In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [12]:
# Creamos un modelo 

class ConvBlock(nn.Module):

    def __init__(self, input_filters, output_size, kernel_size, stride=0, padding=1, num_layers=2):
        super(ConvBlock, self).__init__()
        ConvSeq = []
        input_size = input_filters
        for i in range(num_layers):
            ConvSeq.append(nn.Conv2d(input_size, output_size, kernel_size, stride, padding))
            ConvSeq.append(nn.SELU())
            input_size = output_size
        self.ConvSeq = nn.Sequential(*ConvSeq)

    def forward(self, x):
        return self.ConvSeq(x)
    
class MLRS(nn.Module):

    def __init__(self, input_size, output_size, hidden_size, num_filters, Sinput_size):
        super(MLRS, self).__init__()
        self.input_size = input_size
        bs, c, h, w = input_size
        self.Sinput_size = Sinput_size
        self.output_size = output_size
        self.ConvBlock1 = ConvBlock(c, num_filters, kernel_size=3, stride=1, padding=1)# (bs, num_filters, h, w) # (bs, num_filters, 256, 256)
        self.MaxPool1 = nn.MaxPool2d(kernel_size=2, stride=2) # (bs, num_filters, h/2, w/2) # (bs, num_filters, 128, 128)
        self.ConvBlock2 = ConvBlock(num_filters, 2*num_filters, kernel_size=3, stride=1, padding=1) # (bs, 2*num_filters, h/2, w/2) # (bs, 2*num_filters, 128, 128)
        self.MaxPool2 = nn.MaxPool2d(kernel_size=2, stride=2) # (bs, 2*num_filters, h/4, w/4) # (bs, 2*num_filters, 64, 64)
        self.ConvBlock3 = ConvBlock(2*num_filters, 4*num_filters, kernel_size=3, stride=1, padding=1) # (bs, 4*num_filters, h/4, w/4) # (bs, 4*num_filters, 64, 64)
        self.MaxPool3 = nn.MaxPool2d(kernel_size=2, stride=2) # (bs, 4*num_filters, h/8, w/8) # (bs, 4*num_filters, 32, 32)
        self.ConvBlock4 = ConvBlock(4*num_filters, 8*num_filters, kernel_size=3, stride=1, padding=1) # (bs, 8*num_filters, h/8, w/8) # (bs, 8*num_filters, 32, 32)
        self.MaxPool4 = nn.MaxPool2d(kernel_size=2, stride=2) # (bs, 8*num_filters, h/16, w/16) # (bs, 8*num_filters, 16, 16)
        self.ConvBlock5 = ConvBlock(8*num_filters, 16*num_filters, kernel_size=3, stride=1, padding=1) # (bs, 16*num_filters, h/16, w/16) # (bs, 16*num_filters, 16, 16)
        self.MaxPool5 = nn.MaxPool2d(kernel_size=2, stride=2) # (bs, 16*num_filters, h/32, w/32) # (bs, 16*num_filters, 8, 8)

        # Fully Connected Layers
        self.CoordFC = nn.Linear(Sinput_size, hidden_size)
        self.FC1 = nn.Linear(16*num_filters*8*8, hidden_size)
        self.FC2 = nn.Linear(2*hidden_size, hidden_size)
        self.FC3 = nn.Linear(hidden_size, output_size)

    def forward(self, x, Sinput):
        bs, c, h, w = x.size()
        x = self.ConvBlock1(x)
        x = self.MaxPool1(x)
        x = self.ConvBlock2(x)
        x = self.MaxPool2(x)
        x = self.ConvBlock3(x)
        x = self.MaxPool3(x)
        x = self.ConvBlock4(x)
        x = self.MaxPool4(x)
        x = self.ConvBlock5(x)
        x = self.MaxPool5(x)
        x = x.view(bs, -1)
        x = self.FC1(x)
        x = torch.relu(x)
        Sinput = self.CoordFC(Sinput)
        x = torch.cat((x, Sinput), dim=1)
        x = self.FC2(x)
        x = torch.relu(x)
        x = self.FC3(x)
        x = torch.sigmoid(x)
        return x

In [16]:
# Creamos un modelo
model = MLRS(input_size=(1, 18, 256, 256), output_size=17, hidden_size=100, num_filters=16, Sinput_size=17)
# Creamos variables dummy
x = torch.randn(1, 18, 256, 256)
Sinput = torch.randn(1, 17)
# Corremos el modelo
y = model(x, Sinput)
# Imprimimos el resultado
print(y.shape)

torch.Size([1, 17])


In [23]:
# Creamos un dataset Dummy
X = torch.randn(100,18, 256, 256)
Sinput = torch.randn(100, 17)
Y = torch.randn(100, 17)
# Creamos las funciones de perdida y optimizador
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [24]:
import time

In [25]:
# Creamos el loop de entrenamiento
# Creamos un contador de tiempo
start_time = time.time()
for epoch in range(100):
    epoch_time = time.time()
    # Forward pass
    y_pred = model(X, Sinput)
    # Compute Loss
    loss = criterion(y_pred, Y)
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Zero gradients, backward pass, update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("--- %s seconds ---" % (time.time() - epoch_time))
# Imprimimos el tiempo que tomo el entrenamiento
print("--- %s seconds ---" % (time.time() - start_time))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1638400 and 16384x100)